# **Import the Libraries**

In [1]:
!ps -aux|grep python

root          66  0.1  0.0      0     0 ?        Z    16:22   0:15 [python3] <defunct>
root          67  0.0  0.4  77384 57704 ?        S    16:22   0:00 python3 /usr/local/bin/colab-fileshim.py
root         116  0.0  1.0 386544 136052 ?       Sl   16:22   0:08 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root        4617  0.0  0.0   4364  3120 ?        S    16:40   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root        4620  0.0  0.0  20884 12856 ?        S    16:40   0:00 python3 /opt/google/drive/drive-filter.py
root       23446  0.0  0.1 1241404 17860 ?       Sl   17:53   0:00 /usr/colab/bin/language_service --lsp_search_dirs=/datalab/web/pyright/typeshed-fallback/stdlib,/usr/local/lib/python3.10/dist-pa

In [2]:
import os
os.system("kill -9 1")


0

In [3]:
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
# importing required libraries for normalizing data
from sklearn.preprocessing import StandardScaler,LabelBinarizer,MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
# representation of model layers
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import joblib
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB

# **Visualisation**

In [4]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))
if not tf.config.list_physical_devices('GPU'):
    print("⚠️ Không tìm thấy GPU! Hãy kiểm tra cài đặt Kaggle Notebook.")
else:
    print("✅ GPU đã sẵn sàng để sử dụng!")

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ GPU đã sẵn sàng để sử dụng!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

path = "/content/drive/MyDrive/Do_An_Tot_Nghiep/dataset/CICIoT2023/part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"
print(os.path.exists(path))

True


In [4]:
pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.0 MB/s eta 0:00:00


In [6]:
import os
import cudf  # Sử dụng cuDF để tăng tốc xử lý dữ liệu bằng GPU
import matplotlib.pyplot as plt

# 📂 Thư mục chứa dữ liệu
data_dir = "/content/drive/MyDrive/Do_An_Tot_Nghiep/dataset/CICIoT2023/"

# 🛠 Đọc các file CSV vào danh sách
dfs = []
for i in range(0, 10):
    filename = f"{data_dir}part-{str(i).zfill(5)}-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"

    if os.path.exists(filename):
        print(f"📂 Đọc file: {filename}")
        df = cudf.read_csv(filename)
        dfs.append(df)
    else:
        print(f"⚠ File không tồn tại: {filename}")

# 🧪 Load file test (part-00168)
test_filename = f"{data_dir}part-00168-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"
if os.path.exists(test_filename):
    print(f"📂 Đọc file test: {test_filename}")
    df_test = cudf.read_csv(test_filename)
else:
    print(f"⚠ File test không tồn tại: {test_filename}")

# ✅ Nối dữ liệu nếu có
if dfs:
    df_full = cudf.concat(dfs, ignore_index=True)
    print("✅ Dữ liệu đã được nối thành công!")
else:
    print("❌ Không có dữ liệu nào để nối!")
    df_full = None

# 📊 Kiểm tra nhãn gốc trước khi gộp
if df_full is not None and "label" in df_full.columns:
    # Chuyển cuDF sang Pandas để vẽ biểu đồ
    df_pandas = df_full.to_pandas()

    print("📋 Danh sách nhãn gốc trong dữ liệu:")
    print(df_pandas["label"].value_counts())

    # 📊 Vẽ biểu đồ phân phối nhãn gốc
    fig, ax = plt.subplots(figsize=(15, 9))
    label_counts = df_pandas["label"].value_counts()

    bars = ax.bar(label_counts.index, label_counts.values, width=0.8)

    plt.xticks(rotation=45, ha='right')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
    ax.set_xlabel('Labels')
    ax.set_ylabel('Count')
    ax.set_title('Original Label Distribution')

    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{int(height):,}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
else:
    print("⚠ Không thể kiểm tra nhãn vì dữ liệu rỗng hoặc thiếu cột 'label'!")


📂 Đọc file: /content/drive/MyDrive/Do_An_Tot_Nghiep/dataset/CICIoT2023/part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


MemoryError: std::bad_alloc: CUDA error at: /pyenv/versions/3.12.9/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version

In [ ]:
import os
import cudf  # Sử dụng cuDF để tăng tốc xử lý dữ liệu bằng GPU
import matplotlib.pyplot as plt

# 📂 Thư mục chứa dữ liệu
data_dir = "/content/drive/MyDrive/Do_An_Tot_Nghiep/dataset/CICIoT2023/"

# 🛠 Đọc các file CSV vào danh sách
dfs = []
for i in range(0, 10):
    filename = f"{data_dir}part-{str(i).zfill(5)}-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"

    if os.path.exists(filename):
        print(f"📂 Đọc file: {filename}")
        df = cudf.read_csv(filename)
        dfs.append(df)
    else:
        print(f"⚠ File không tồn tại: {filename}")

# 🧪 Load file test (part-00168)
test_filename = f"{data_dir}part-00168-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"
if os.path.exists(test_filename):
    print(f"📂 Đọc file test: {test_filename}")
    df_test = cudf.read_csv(test_filename)
else:
    print(f"⚠ File test không tồn tại: {test_filename}")

# ✅ Nối dữ liệu nếu có
if dfs:
    df_full = cudf.concat(dfs, ignore_index=True)
    print("✅ Dữ liệu đã được nối thành công!")
else:
    print("❌ Không có dữ liệu nào để nối!")
    df_full = None

# 📊 Vẽ biểu đồ phân bố nhãn nếu dữ liệu hợp lệ
if df_full is not None and "label" in df_full.columns:
    # Chuyển cuDF sang Pandas để vẽ biểu đồ
    df_filtered_pandas = df_full.to_pandas()

    fig, ax = plt.subplots(figsize=(15, 9))
    label_counts = df_filtered_pandas["label"].value_counts()

    bars = ax.bar(label_counts.index, label_counts.values, width=0.8)

    # 🎨 Định dạng biểu đồ
    plt.xticks(rotation=45, ha='right')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
    ax.set_xlabel('Labels')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Labels')

    # 🏷 Hiển thị số lượng trên cột
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{int(height):,}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
else:
    print("⚠ Không thể vẽ biểu đồ vì df_full rỗng hoặc thiếu cột 'label'!")


In [ ]:
df.info()

In [ ]:
# Danh sách giao thức
protocols = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

# Tính tổng số lần xuất hiện của mỗi giao thức
protocol_counts = df[protocols].sum().sort_values(ascending=False)

# Chuyển từ cuDF Series → Pandas Series
protocol_counts_pandas = protocol_counts.to_pandas()

# Vẽ biểu đồ
plt.figure(figsize=(14, 8))
protocol_counts_pandas.plot(kind='bar')
plt.title('Protocol Usage Counts')
plt.xlabel('Protocol')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')  # Xoay nhãn trục X
plt.tight_layout()  # Căn chỉnh bố cục
plt.show()

In [ ]:
# Tính toán các chỉ số
average_flow_duration = df['flow_duration'].mean()
max_protocol_usage = df[protocols].sum().max()

# Chuyển cuDF Series sang Pandas trước khi gọi idxmax()
top_attack_type = df['label'].value_counts().to_pandas().idxmax()

# Tạo figure
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Network Traffic Scorecard', fontsize=20)

# Scorecard cho thời gian trung bình của luồng dữ liệu
axs[0].text(0.5, 0.5, f'Avg Flow Duration\n{average_flow_duration:.2f} sec',
            fontsize=15, ha='center', va='center')
axs[0].set_axis_off()

# Scorecard cho giao thức phổ biến nhất
axs[1].text(0.5, 0.5, f'Max Protocol Usage\n{max_protocol_usage}',
            fontsize=15, ha='center', va='center')
axs[1].set_axis_off()

# Scorecard cho kiểu tấn công phổ biến nhất
axs[2].text(0.5, 0.5, f'Top Attack Type\n{top_attack_type}',
            fontsize=15, ha='center', va='center')
axs[2].set_axis_off()

# Hiển thị biểu đồ
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cudf  # Dùng cuDF để xử lý dữ liệu nhanh hơn trên GPU

# 🟢 Hàm thay đổi nhãn về 8 nhóm chính (Đã cập nhật đầy đủ nhãn)
def change_label(df):
    mapping = {
        'DDoS-ICMP_Flood': 'DDos', 'DDoS-UDP_Flood': 'DDos', 'DDoS-TCP_Flood': 'DDos',
        'DDoS-PSHACK_Flood': 'DDos', 'DDoS-SYN_Flood': 'DDos', 'DDoS-RSTFINFlood': 'DDos',
        'DDoS-SynonymousIP_Flood': 'DDos', 'DDoS-ICMP_Fragmentation': 'DDos',
        'DDoS-UDP_Fragmentation': 'DDos', 'DDoS-ACK_Fragmentation': 'DDos',
        'DDoS-HTTP_Flood': 'DDos', 'DDoS-SlowLoris': 'DDos',

        'DoS-UDP_Flood': 'DoS', 'DoS-TCP_Flood': 'DoS', 'DoS-SYN_Flood': 'DoS', 'DoS-HTTP_Flood': 'DoS',

        'Recon-HostDiscovery': 'Recon', 'Recon-OSScan': 'Recon', 'Recon-PortScan': 'Recon',
        'Recon-PingSweep': 'Recon', 'VulnerabilityScan': 'Recon',

        'MITM-ArpSpoofing': 'Spoofing', 'DNS_Spoofing': 'Spoofing',

        'DictionaryBruteForce': 'BruteForce',

        'BrowserHijacking': 'Web-based', 'XSS': 'Web-based', 'Uploading_Attack': 'Web-based',
        'SqlInjection': 'Web-based', 'CommandInjection': 'Web-based', 'Backdoor_Malware': 'Web-based',

        'Mirai-greeth_flood': 'Mirai', 'Mirai-udpplain': 'Mirai', 'Mirai-greip_flood': 'Mirai',

        'BenignTraffic': 'BENIGN'
    }

    df["label"] = df["label"].map(mapping).fillna(df["label"])  # Chỉ thay đổi nhãn có trong mapping

# 🟢 Đọc dữ liệu từ thư mục Kaggle
data_dir = "/content/drive/MyDrive/Do_An_Tot_Nghiep/dataset/CICIoT2023/"

# 🛠 Đọc các file CSV vào danh sách
dfs = []
for i in range(0, 10):
    filename = f"{data_dir}part-{str(i).zfill(5)}-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"

    if os.path.exists(filename):
        print(f"📂 Đọc file: {filename}")
        df = cudf.read_csv(filename)
        dfs.append(df)
    else:
        print(f"⚠ File không tồn tại: {filename}")

# ✅ Nối dữ liệu nếu có
if dfs:
    df_full = cudf.concat(dfs, ignore_index=True)
    print("✅ Dữ liệu đã được nối thành công!")
else:
    print("❌ Không có dữ liệu nào để nối!")
    df_full = None

# 🟢 Áp dụng gộp nhãn nếu dữ liệu hợp lệ
if df_full is not None and "label" in df_full.columns:
    change_label(df_full)

    # 🟢 Chuyển về Pandas để kiểm tra & vẽ biểu đồ
    df_pandas = df_full.to_pandas()

    # 🟢 Kiểm tra danh sách nhãn sau khi gộp
    print("\n📋 Danh sách nhãn sau khi gộp:")
    print(df_pandas["label"].value_counts())

    # 🟢 Kiểm tra xem có nhãn nào chưa được gộp
    valid_labels = {'DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN'}
    unmapped_labels = set(df_pandas["label"].unique()) - valid_labels

    print("\n🔍 Nhãn chưa được gộp:")
    print(unmapped_labels if unmapped_labels else "✅ Không có nhãn nào bị bỏ sót!")

    # 🟢 Vẽ biểu đồ phân bố nhãn sau khi gộp
    fig, ax = plt.subplots(figsize=(10, 5))
    label_counts = df_pandas["label"].value_counts()

    bars = ax.bar(label_counts.index, label_counts.values, width=0.5)
    plt.xticks(rotation=45, ha='right')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
    ax.set_xlabel('Labels')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Labels After Merging')

    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{int(height):,}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

else:
    print("⚠ Không thể thực hiện gộp nhãn vì dữ liệu rỗng hoặc thiếu cột 'label'!")


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import numpy as np

# 🟢 Chuyển cuDF sang Pandas nếu cần
if hasattr(df_full, "to_pandas"):
    df_balanced = df_full.to_pandas()
else:
    df_balanced = df_full.copy()

# 🟢 Lấy nhãn và dữ liệu
X = df_balanced.drop(columns=['label'])  # Các đặc trưng
y = df_balanced['label']  # Nhãn

# 📊 Xem phân phối trước khi cân bằng
print("\n📊 Phân phối nhãn ban đầu:")
print(y.value_counts())

# 🟢 Xác định số lượng mẫu mục tiêu cho mỗi lớp
target_counts = y.value_counts()

# 🛠 **Tối ưu chiến lược cân bằng**
max_threshold = int(target_counts.max() * 0.2)  # Giảm lớp lớn nhất xuống 20%
min_threshold = int(target_counts.min() * 15)  # Tăng lớp nhỏ nhất lên 15 lần

sampling_strategy_under = {cls: max_threshold for cls in target_counts.index if target_counts[cls] > max_threshold}
sampling_strategy_over = {cls: min_threshold for cls in target_counts.index if target_counts[cls] < min_threshold}

# 🏗 Pipeline kết hợp **Undersampling + SMOTE**
pipeline = Pipeline([
    ('undersample', RandomUnderSampler(sampling_strategy=sampling_strategy_under, random_state=42)),
    ('smote', SMOTE(sampling_strategy=sampling_strategy_over, random_state=42))
])

# 🚀 Cân bằng dữ liệu
X_resampled, y_resampled = pipeline.fit_resample(X, y)

# 🟢 Tạo DataFrame mới sau cân bằng
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['label'] = y_resampled

# 📊 Kiểm tra lại phân phối nhãn sau khi cân bằng
print("\n📊 Phân phối nhãn sau cân bằng:")
print(df_resampled["label"].value_counts())

# 📊 Vẽ biểu đồ phân phối sau khi cân bằng
fig, ax = plt.subplots(figsize=(10, 5))
label_counts = df_resampled["label"].value_counts()

bars = ax.bar(label_counts.index, label_counts.values, width=0.5)
plt.xticks(rotation=45, ha='right')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
ax.set_xlabel('Labels')
ax.set_ylabel('Count')
ax.set_title('Distribution of Labels After Improved Balancing')

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{int(height):,}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# Hàm thay đổi nhãn về 7 nhóm chính
def change_label(df):
    df.label.replace([

        'DDoS-SYN_Flood','DDoS-RSTFINFlood','DDoS-SynonymousIP_Flood','DDoS-ICMP_Fragmentation',
        'DDoS-UDP_Fragmentation','DDoS-ACK_Fragmentation','DDoS-HTTP_Flood','DDoS-SlowLoris'
    ], 'DDos', inplace=True)

    df.label.replace(['DoS-UDP_Flood','DoS-TCP_Flood','DoS-SYN_Flood','DoS-HTTP_Flood'], 'DoS', inplace=True)
    df.label.replace(['Recon-HostDiscovery','Recon-OSScan','Recon-PortScan','Recon-PingSweep','VulnerabilityScan'], 'Recon', inplace=True)
    df.label.replace(['MITM-ArpSpoofing','DNS_Spoofing'], 'Spoofing', inplace=True)
    df.label.replace(['DictionaryBruteForce'], 'BruteForce', inplace=True)
    df.label.replace(['BrowserHijacking','XSS','Uploading_Attack','SqlInjection','CommandInjection','Backdoor_Malware'], 'Web-based', inplace=True)
    df.label.replace(['Mirai-greeth_flood','Mirai-udpplain','Mirai-greip_flood'], 'Mirai', inplace=True)
    df.label.replace(['BenignTraffic'], 'BENIGN', inplace=True)

# Thay đổi nhãn trong dataset
change_label(df)
change_label(df_test)

# Chọn các lớp đã được gom nhóm
selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
df_selected = df[df['label'].isin(selected_classes)]
df_selected_test = df_test[df_test['label'].isin(selected_classes)]

# Chuyển cuDF sang Pandas trước khi xử lý
if hasattr(df_selected, "to_pandas"):
    df_selected = df_selected.to_pandas()

# Áp dụng SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(df_selected.drop(columns=['label']).to_numpy(), df_selected['label'].to_numpy())

# Tạo dataframe mới sau khi SMOTE
df_balanced = pd.DataFrame(X_resampled, columns=df_selected.drop(columns=['label']).columns)
df_balanced['label'] = y_resampled

# Vẽ biểu đồ phân phối sau khi SMOTE
x_label_distribute = np.array(df_balanced['label'].value_counts().index)
y_label_distribute = np.array(df_balanced['label'].value_counts().values)

fig = plt.figure(figsize=(10, 5))
bars = plt.bar(x_label_distribute, y_label_distribute, width=0.5, align='center')
plt.bar_label(bars, label_type='edge', fontsize=10)
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.xlabel('Labels')
plt.ylabel('Count')
plt.title('Distribution of Labels After SMOTE')
plt.tight_layout()
plt.show()

# Kiểm tra lại phân phối
print(df_balanced['label'].value_counts())


In [ ]:
print(type(df))  # Kiểm tra df có phải cuDF không
print(df["label"].unique())  # Kiểm tra nhãn có đủ không


# **Transformation and standardization**

In [ ]:
# Chuyển cột label từ cuDF sang pandas
label = pd.DataFrame(df["label"].to_pandas())

# Hiển thị dữ liệu
print(label.head())

In [ ]:
# Scale data will convert scale data or data format to the same range to avoid too large data affecting the model
# Scale data for each numeric feature with the train dataset
# Scale using MinMaxScaler to avoid negative values
# Here we only need to scale numeric features and numeric attributes because only numeric attributes have a large range
def scaleStandardData(dataFrame, numeric_cols):
  scaler = preprocessing.StandardScaler()
  for col in numeric_cols:
    arr = dataFrame[col]
    arr = np.array(arr)
    dataFrame[col] = scaler.fit_transform(arr.reshape(len(arr),1))
  return dataFrame

def scaleMinMaxData(dataFrame, numeric_cols):
  scaler = preprocessing.MinMaxScaler()
  for col in numeric_cols:
    arr = dataFrame[col]
    arr = np.array(arr)
    dataFrame[col] = scaler.fit_transform(arr.reshape(len(arr),1))
  return dataFrame

def scaleData(dataFrame, numeric_cols):
  dataFrame = scaleStandardData(dataFrame, numeric_cols)
  dataFrame = scaleMinMaxData(dataFrame, numeric_cols)
  return dataFrame

In [ ]:
Y_TRAIN = df_selected['label']
Y_TEST = df_selected_test['label']
X_TRAIN = df_selected.drop('label', axis=1).copy()
X_TEST = df_selected_test.drop('label', axis=1).copy()

import numpy as np

# Kiểm tra và chuyển đổi tất cả các cột float64 thành float64 (tránh float128)
float_cols = X_TRAIN.select_dtypes(include=['float64']).columns
X_TRAIN[float_cols] = X_TRAIN[float_cols].astype(np.float64)

float_cols = X_TEST.select_dtypes(include=['float64']).columns
X_TEST[float_cols] = X_TEST[float_cols].astype(np.float64)

In [ ]:
# Tạo từ điển ánh xạ
label_mapping = {
    'BENIGN': 0, 'DDos': 1, 'DoS': 2, 'Mirai': 3,
    'Spoofing': 4, 'Recon': 5, 'Web-based': 6, 'BruteForce': 7
}

# Sử dụng map() để thay thế nhãn
Y_TRAIN = Y_TRAIN.map(label_mapping)
Y_TEST = Y_TEST.map(label_mapping)

In [ ]:
print(Y_TRAIN.unique())  # Kiểm tra xem có giá trị NaN không
print(Y_TRAIN.isnull().sum())  # Nếu có NaN, sửa lỗi trước khi train


In [ ]:
print(X_TRAIN.describe())  # Kiểm tra xem dữ liệu có bị biến đổi bất thường không
print(X_TEST.describe())


# **Feature Selection**

In [ ]:
import cudf

# Kiểm tra nếu X_TRAIN là cudf.DataFrame thì mới chuyển đổi
if isinstance(X_TRAIN, cudf.DataFrame):
    X_TRAIN_pd = X_TRAIN.to_pandas()
else:
    X_TRAIN_pd = X_TRAIN  # Nếu không phải cudf thì giữ nguyên

# Lấy danh sách cột số
numeric_features = X_TRAIN_pd.select_dtypes(include='number').columns

# Thực hiện scale dữ liệu
X_TRAIN_scaled = scaleData(X_TRAIN_pd, numeric_features)

# Nếu ban đầu là cudf, chuyển ngược lại
if isinstance(X_TRAIN, cudf.DataFrame):
    X_TRAIN = cudf.from_pandas(X_TRAIN_scaled)
else:
    X_TRAIN = X_TRAIN_scaled


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

# Kiểm tra nếu X_TRAIN là cudf.DataFrame thì chuyển về pandas, nếu không thì giữ nguyên
if hasattr(X_TRAIN, "to_pandas"):
    X_TRAIN_pd = X_TRAIN.to_pandas()
else:
    X_TRAIN_pd = X_TRAIN

if hasattr(Y_TRAIN, "to_pandas"):
    Y_TRAIN_pd = Y_TRAIN.to_pandas()
else:
    Y_TRAIN_pd = Y_TRAIN

# Tiến hành chọn lọc đặc trưng bằng RFE
clf = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf, n_features_to_select=30, step=1)
rfe.fit(X_TRAIN_pd, Y_TRAIN_pd.astype(int))

X_rfeTrain = rfe.transform(X_TRAIN_pd)
true = rfe.support_
rfecolindex_train = [i for i, x in enumerate(true) if x]
rfecolname_train = [numeric_features[i] for i in rfecolindex_train]


In [ ]:
X_TRAIN.head()

In [ ]:
# Kiểm tra nếu X_TRAIN, Y_TRAIN đang là cudf thì chuyển sang pandas
if isinstance(X_TRAIN, cudf.DataFrame):
    X_TRAIN = X_TRAIN.to_pandas()
if isinstance(Y_TRAIN, cudf.Series):
    Y_TRAIN = Y_TRAIN.to_pandas()

# Chuyển Y_TRAIN về kiểu int (nếu chưa đúng)
Y_TRAIN = Y_TRAIN.astype(int)

# Kiểm tra và xử lý dữ liệu NaN (nếu có)
X_TRAIN = X_TRAIN.fillna(X_TRAIN.mean())

# Chỉ giữ lại các cột số (loại bỏ object nếu có)
X_TRAIN = X_TRAIN.select_dtypes(include=['number'])

# Danh sách các cột số
numeric_features = X_TRAIN.columns

# Khởi tạo mô hình Decision Tree
clf = DecisionTreeClassifier(random_state=0)

# Áp dụng Recursive Feature Elimination (RFE)
rfe = RFE(estimator=clf, n_features_to_select=30, step=1)
rfe.fit(X_TRAIN, Y_TRAIN)

# Trích xuất các đặc trưng quan trọng
X_rfeTrain = rfe.transform(X_TRAIN)
true = rfe.support_
rfecolindex_train = [i for i, x in enumerate(true) if x]
rfecolname_train = list(numeric_features[i] for i in rfecolindex_train)

# In ra danh sách các cột được chọn
print("Các cột được chọn:", rfecolname_train)

# List of selected important features
selected_features = ['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 'syn_flag_number', 'psh_flag_number', 'ack_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'UDP', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight']

# Select important features from the training set
X_rfeTrain = X_TRAIN.loc[:, selected_features]
X_TEST = X_TEST.loc[:, selected_features]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_rfeTrain, Y_TRAIN, test_size=0.2, random_state=42)

In [ ]:
# Data set size
sizes = [X_train.shape[0], X_test.shape[0], Y_train.shape[0], Y_test.shape[0]]
labels = ['X_train', 'X_test', 'Y_train', 'Y_test']

# Draw a column chart
plt.figure(figsize=(8, 6))
bars = plt.bar(labels, sizes, color=['blue', 'grey', 'blue', 'grey'])
plt.xlabel('Dataset')
plt.ylabel('Number of samples')
plt.title('Size of the datasets')

# Display data at the top of the column
for bar, size in zip(bars, sizes):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(size), ha='center', va='bottom', fontsize=10)

plt.show()

In [ ]:
print(X_train.shape)
print(X_TEST.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# ✅ **1. Chia tập train/test đảm bảo ngẫu nhiên và đồng đều**
df_train, df_test = train_test_split(df_resampled, test_size=0.2, random_state=42, stratify=df_resampled['label'])

# ✅ **2. Tách nhãn và đặc trưng**
Y_TRAIN = df_train['label'].copy()
Y_TEST = df_test['label'].copy()
X_TRAIN = df_train.drop('label', axis=1).copy()
X_TEST = df_test.drop('label', axis=1).copy()

# ✅ **3. Chuyển đổi nhãn thành số**
label_mapping = {'BENIGN': 0, 'DDos': 1, 'DoS': 2, 'Mirai': 3,
                 'Spoofing': 4, 'Recon': 5, 'Web-based': 6, 'BruteForce': 7}
Y_TRAIN = Y_TRAIN.map(label_mapping).astype(int)
Y_TEST = Y_TEST.map(label_mapping).astype(int)

# ✅ **4. Xử lý giá trị NaN (dùng giá trị trung vị để tránh ảnh hưởng của ngoại lai)**
imputer = SimpleImputer(strategy="median")
X_TRAIN = pd.DataFrame(imputer.fit_transform(X_TRAIN), columns=X_TRAIN.columns)
X_TEST = pd.DataFrame(imputer.transform(X_TEST), columns=X_TEST.columns)

# ✅ **5. Scale dữ liệu đúng cách (dùng RobustScaler để tránh ảnh hưởng của giá trị ngoại lai)**
scaler = RobustScaler()
X_TRAIN = pd.DataFrame(scaler.fit_transform(X_TRAIN), columns=X_TRAIN.columns)
X_TEST = pd.DataFrame(scaler.transform(X_TEST), columns=X_TEST.columns)

# ✅ **6. Chọn lọc đặc trưng tốt hơn bằng RandomForest**
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), max_features=30)
selector.fit(X_TRAIN, Y_TRAIN)

selected_features = X_TRAIN.columns[selector.get_support()]
print(f"✅ {len(selected_features)} Features Selected: {list(selected_features)}")

X_TRAIN = X_TRAIN[selected_features]
X_TEST = X_TEST[selected_features]

# ✅ **7. Kiểm tra lại kích thước tập dữ liệu**
sizes = [X_TRAIN.shape[0], X_TEST.shape[0], Y_TRAIN.shape[0], Y_TEST.shape[0]]
labels = ['X_train', 'X_test', 'Y_train', 'Y_test']

plt.figure(figsize=(8, 6))
bars = plt.bar(labels, sizes, color=['blue', 'grey', 'blue', 'grey'])

# 🏷️ **Hiển thị số lượng mẫu trên biểu đồ**
for bar, size in zip(bars, sizes):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(size), ha='center', va='bottom', fontsize=10)

plt.xlabel('Dataset')
plt.ylabel('Number of samples')
plt.title('Size of the datasets')
plt.show()


# **Models**

In [ ]:
# pip install torch torchvision torchaudio

# **FT Trasformer**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# 🛑 Giải phóng bộ nhớ GPU nếu cần
torch.cuda.empty_cache()

# 1️⃣ Kiểm tra kích thước dữ liệu gốc trước khi chuyển đổi
print("🔍 Checking dataset shapes before processing...")
print("X_TRAIN shape:", X_TRAIN.shape)
print("Y_TRAIN shape:", Y_TRAIN.shape)
print("X_TEST shape:", X_TEST.shape)
print("Y_TEST shape:", Y_TEST.shape)

# 2️⃣ Kiểm tra và xử lý NaN (nếu có)
X_TRAIN = X_TRAIN.fillna(0)
X_TEST = X_TEST.fillna(0)

# 3️⃣ Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_TRAIN.values)  # ✅ Sửa: Dùng X_TRAIN đúng cách
X_test_np = scaler.transform(X_TEST.values)

# 4️⃣ Chuyển nhãn sang NumPy và đảm bảo đúng định dạng
Y_train_np = Y_TRAIN.values.squeeze()  # ✅ Sửa: Dùng .squeeze() để loại bỏ chiều thừa
Y_test_np = Y_TEST.values.squeeze()

# 5️⃣ Chuyển dữ liệu thành Tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long)

# 6️⃣ Kiểm tra kích thước sau khi chuyển đổi
print("\n🔍 Checking tensor shapes after processing...")
print("X_train_tensor shape:", X_train_tensor.shape)
print("Y_train_tensor shape:", Y_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)
print("Y_test_tensor shape:", Y_test_tensor.shape)

# 🚀 Kiểm tra lại số lượng mẫu khớp nhau trước khi tiếp tục
assert X_train_tensor.shape[0] == Y_train_tensor.shape[0], "❌ Mismatch between X_train and Y_train!"
assert X_test_tensor.shape[0] == Y_test_tensor.shape[0], "❌ Mismatch between X_test and Y_test!"

# 7️⃣ Định nghĩa mô hình FT-Transformer
class FTTransformer(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64, num_heads=4, num_blocks=3):
        super(FTTransformer, self).__init__()

        # Embedding đặc trưng
        self.feature_embedding = nn.Linear(num_features, embed_dim)

        # Positional Encoding
        self.positional_encoding = nn.Parameter(torch.randn(1, num_features, embed_dim))

        # Transformer Encoder
        self.transformer_blocks = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True),
            num_layers=num_blocks
        )

        # Mạng phân loại đầu ra
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        batch_size, num_features = x.shape

        # Positional encoding phù hợp kích thước
        pos_enc = self.positional_encoding[:, :num_features, :].expand(batch_size, -1, -1)

        # Áp dụng embedding và positional encoding
        x = self.feature_embedding(x).unsqueeze(1) + pos_enc
        x = self.transformer_blocks(x)  # Transformer Encoder
        x = x.mean(dim=1)  # Lấy trung bình theo feature
        x = self.classifier(x)  # Dự đoán
        return x

# 8️⃣ Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_train_tensor.shape[1]
num_classes = len(np.unique(Y_train_np))

model = FTTransformer(num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# 9️⃣ Tạo Dataloader
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

# 🔍 Kiểm tra lại batch size
print(f"\n✅ Training batches: {len(train_loader)}, Test batches: {len(test_loader)}")

# 🔟 Huấn luyện mô hình với Early Stopping
num_epochs = 50
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    # Đánh giá trên tập kiểm tra
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    # Early Stopping
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

# 🔟 Đánh giá cuối cùng
print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")


In [ ]:
import gc
import torch

def report_gpu():
    print(torch.cuda.list_gpu_processes())  # Hiển thị danh sách các tiến trình GPU
    gc.collect()  # Thu hồi bộ nhớ rác
    torch.cuda.empty_cache()  # Giải phóng bộ nhớ GPU

# Gọi hàm để kiểm tra GPU
report_gpu()


In [ ]:
import gc
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🧹 Giải phóng bộ nhớ GPU trước khi chạy
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    print(torch.cuda.list_gpu_processes())

clear_gpu()

# 🔄 Chuyển dữ liệu về thiết bị
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)
model.to(device)

# 🚀 Chạy mô hình trên mini-batch
batch_size = 512
y_scores_list, y_pred_list, y_true_list = [], [], []

model.eval()
with torch.no_grad():
    for i in range(0, len(X_test_tensor), batch_size):
        X_batch = X_test_tensor[i:i+batch_size]
        Y_batch = Y_test_tensor[i:i+batch_size]

        with torch.amp.autocast(device_type='cuda'):
            logits = model(X_batch)
            y_scores_batch = F.softmax(logits, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

# Gộp lại kết quả
y_scores = np.vstack(y_scores_list)
y_pred = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Danh sách nhãn
selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
num_classes = len(selected_classes)

# 📌 Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# 📌 Cải tiến ROC-AUC (Giới hạn trục X và Y)
def plot_roc_auc(y_true, y_scores, class_names):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 0.02])  # Giới hạn X từ 0 - 0.02
    plt.ylim([0.5, 1])   # Giới hạn Y từ 0.5 - 1
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Zoomed In)')
    plt.legend()
    plt.grid(True)
    plt.show()

# 📌 Cải tiến PCA & t-SNE (3D) với chú thích màu sắc
# Hàm vẽ PCA và t-SNE 3D
def plot_tsne_pca_3d(X, y, class_names):
    sample_size = min(5000, len(X))  # Chọn tối đa 5000 mẫu để vẽ
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    # Giảm chiều với PCA và t-SNE
    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")  # Bảng màu phân biệt
    colors = cmap(np.linspace(0, 1, len(class_names)))

    # Vẽ PCA 3D
    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title('PCA 3D Visualization')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    # Vẽ t-SNE 3D
    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title('t-SNE 3D Visualization')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()


# 📌 Vẽ Training & Validation Loss/Accuracy
def plot_training_curves(train_loss, val_loss, train_acc, val_acc):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'r', label='Train Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training & Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training & Validation Accuracy')
    plt.legend()

    plt.show()

# 📊 Gọi các hàm vẽ biểu đồ
plot_confusion_matrix(y_true, y_pred, selected_classes)
plot_roc_auc(y_true, y_scores, selected_classes)
plot_tsne_pca_3d(X_test_tensor.cpu().numpy(), y_true, selected_classes)

# 📌 Nếu có dữ liệu lịch sử huấn luyện, bỏ comment dòng dưới để vẽ đồ thị huấn luyện
#plot_training_curves(train_loss, val_loss, train_acc, val_acc)

# **Tab Tranformer**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 🟢 1️⃣ Load và chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_TRAIN.values)
X_test_np = scaler.transform(X_TEST.values)

# 🔴 Đảm bảo lấy đúng dữ liệu nhãn (chứ không phải dữ liệu đầu vào)
Y_train_np = Y_TRAIN.values  # Nhãn đúng
Y_test_np = Y_TEST.values  # Nhãn đúng

# 🟢 2️⃣ Chuyển sang Tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)

# 🔴 Đảm bảo `Y_train_tensor` có dạng `(batch_size,)`
Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long).squeeze()
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long).squeeze()

# 🟢 3️⃣ Định nghĩa lớp Transformer cải tiến
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.GELU(),
            nn.Linear(embed_dim * 2, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x

# 🟢 4️⃣ Mô hình TabTransformer
class TabTransformer(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64, num_heads=4, num_blocks=3):
        super(TabTransformer, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(num_features, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU()
        )
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(embed_dim, num_heads) for _ in range(num_blocks)]
        )
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Thêm batch dimension
        x = self.transformer_blocks(x)
        x = x.squeeze(0)  # Bỏ batch dimension
        x = self.classifier(x)
        return x

# 🟢 5️⃣ Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_train_tensor.shape[1]  # Số đặc trưng đầu vào
num_classes = len(np.unique(Y_train_np))  # Số lớp đầu ra

model = TabTransformer(num_features, num_classes).to(device)

criterion = nn.CrossEntropyLoss()  # Không dùng label smoothing để tránh lỗi
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# 🟢 6️⃣ Dataloader
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64, shuffle=False)

# 🟢 7️⃣ Huấn luyện mô hình với Early Stopping
num_epochs = 10
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)

        # 🔴 Đảm bảo đầu ra có shape [batch_size, num_classes]
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        # 🔴 Sửa lỗi tính accuracy
        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    # 🟢 Đánh giá trên tập kiểm tra
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    # 🟢 Early Stopping
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

# 🟢 8️⃣ Đánh giá cuối cùng
print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")


In [ ]:
import gc
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🧹 Giải phóng bộ nhớ GPU trước khi chạy
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    print(torch.cuda.list_gpu_processes())

clear_gpu()

# 🔄 Chuyển dữ liệu về thiết bị
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)
model.to(device)

# 🚀 Chạy mô hình trên mini-batch
batch_size = 512
y_scores_list, y_pred_list, y_true_list = [], [], []

model.eval()
with torch.no_grad():
    for i in range(0, len(X_test_tensor), batch_size):
        X_batch = X_test_tensor[i:i+batch_size]
        Y_batch = Y_test_tensor[i:i+batch_size]

        with torch.amp.autocast(device_type='cuda'):
            logits = model(X_batch)
            y_scores_batch = F.softmax(logits, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

# Gộp lại kết quả
y_scores = np.vstack(y_scores_list)
y_pred = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Danh sách nhãn
selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
num_classes = len(selected_classes)

# 📌 Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# 📌 Cải tiến ROC-AUC (Giới hạn trục X và Y)
def plot_roc_auc(y_true, y_scores, class_names):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 0.02])  # Giới hạn X từ 0 - 0.02
    plt.ylim([0.5, 1])   # Giới hạn Y từ 0.5 - 1
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Zoomed In)')
    plt.legend()
    plt.grid(True)
    plt.show()

# 📌 Cải tiến PCA & t-SNE (3D) với chú thích màu sắc
# Hàm vẽ PCA và t-SNE 3D
def plot_tsne_pca_3d(X, y, class_names):
    sample_size = min(5000, len(X))  # Chọn tối đa 5000 mẫu để vẽ
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    # Giảm chiều với PCA và t-SNE
    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")  # Bảng màu phân biệt
    colors = cmap(np.linspace(0, 1, len(class_names)))

    # Vẽ PCA 3D
    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title('PCA 3D Visualization')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    # Vẽ t-SNE 3D
    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title('t-SNE 3D Visualization')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()


# 📌 Vẽ Training & Validation Loss/Accuracy
def plot_training_curves(train_loss, val_loss, train_acc, val_acc):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'r', label='Train Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training & Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training & Validation Accuracy')
    plt.legend()

    plt.show()

# 📊 Gọi các hàm vẽ biểu đồ
plot_confusion_matrix(y_true, y_pred, selected_classes)
plot_roc_auc(y_true, y_scores, selected_classes)
plot_tsne_pca_3d(X_test_tensor.cpu().numpy(), y_true, selected_classes)

# 📌 Nếu có dữ liệu lịch sử huấn luyện, bỏ comment dòng dưới để vẽ đồ thị huấn luyện
#plot_training_curves(train_loss, val_loss, train_acc, val_acc)


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# Chuyển dữ liệu test về CPU để xử lý
X_test_np = X_test_tensor.cpu().numpy()
Y_test_np = Y_test_tensor.cpu().numpy()

# Chạy mô hình để lấy dự đoán
model.eval()
with torch.no_grad():
    logits = model(X_test_tensor.to(device))
    y_scores = F.softmax(logits, dim=1).cpu().numpy()
    y_pred = np.argmax(y_scores, axis=1)
    y_true = Y_test_np

selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
num_classes = len(selected_classes)

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

def plot_roc_auc(y_true, y_scores, class_names):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

def plot_tsne_pca(X, y, class_names):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=2)
    tsne = TSNE(n_components=2, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7)
    plt.legend(handles=scatter.legend_elements()[0], labels=class_names, title="Classes")
    plt.title('PCA Visualization')

    plt.subplot(1, 2, 2)
    scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis', alpha=0.7)
    plt.legend(handles=scatter.legend_elements()[0], labels=class_names, title="Classes")
    plt.title('t-SNE Visualization')

    plt.show()

def plot_training_curves(train_loss, val_loss, train_acc, val_acc):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'r', label='Train Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training & Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training & Validation Accuracy')
    plt.legend()

    plt.show()

plot_confusion_matrix(y_true, y_pred, selected_classes)
plot_roc_auc(y_true, y_scores, selected_classes)
plot_tsne_pca(X_test_np, y_true, selected_classes)

# Nếu có dữ liệu lịch sử huấn luyện:
#plot_training_curves(train_loss, val_loss, train_acc, val_acc)


# XGBoost và MLP****

In [ ]:
pip install torch-geometric

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb

# 🟢 1️⃣ Load và chuẩn hóa dữ liệu
def convert_to_dataframe(X):
    """ Chuyển đổi dữ liệu sang pandas DataFrame nếu cần """
    if isinstance(X, torch.Tensor):
        X = X.cpu().numpy()  # Chuyển từ Tensor -> NumPy
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)  # Chuyển từ NumPy -> DataFrame
    return X

X_train = convert_to_dataframe(X_train)
X_test = convert_to_dataframe(X_test)

# 🛠 Kiểm tra xem X_train và X_test có dữ liệu không
if X_train.empty or X_test.empty:
    raise ValueError("❌ X_train hoặc X_test đang bị rỗng!")

# Tìm cột chung giữa X_train và X_test
common_columns = X_train.columns.intersection(X_test.columns)
if len(common_columns) == 0:
    raise ValueError("❌ Không có cột chung nào giữa X_train và X_test!")
X_train = X_train[common_columns]
X_test = X_test[common_columns]

# 🛠 Xử lý giá trị NaN
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

# 🛠 Đảm bảo dữ liệu số
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train)
X_test_np = scaler.transform(X_test)

# Chuyển đổi Y_train và Y_test thành NumPy
def convert_labels(Y):
    """ Chuyển đổi Y thành NumPy """
    if isinstance(Y, torch.Tensor):
        return Y.cpu().numpy()
    return np.array(Y)

Y_train_np = convert_labels(Y_train)
Y_test_np = convert_labels(Y_test)

# 🟢 2️⃣ Huấn luyện XGBoost
dtrain = xgb.DMatrix(X_train_np, label=Y_train_np)
dtest = xgb.DMatrix(X_test_np, label=Y_test_np)

xgb_params = {
    'objective': 'multi:softprob',
    'num_class': len(np.unique(Y_train_np)),
    'max_depth': 6,
    'learning_rate': 0.1,
    'tree_method': 'hist',
    'eval_metric': 'mlogloss'
}

xgb_model = xgb.train(xgb_params, dtrain, num_boost_round=100)

# 🟢 3️⃣ Trích xuất đặc trưng từ XGBoost
X_train_xgb = xgb_model.predict(dtrain)
X_test_xgb = xgb_model.predict(dtest)

X_train_tensor = torch.tensor(X_train_xgb, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_xgb, dtype=torch.float32)

Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long).squeeze()
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long).squeeze()

# 🟢 4️⃣ Định nghĩa MLP
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

# 🟢 5️⃣ Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_train_tensor.shape[1]
num_classes = len(np.unique(Y_train_np))

mlp_model = MLP(num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(mlp_model.parameters(), lr=0.001, weight_decay=1e-4)

# 🟢 6️⃣ DataLoader
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64, shuffle=False)

# 🟢 7️⃣ Huấn luyện MLP
num_epochs = 10
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    mlp_model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = mlp_model(X_batch)

        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    mlp_model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = mlp_model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")

In [ ]:
import gc
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🧹 Giải phóng bộ nhớ GPU trước khi chạy
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    print(torch.cuda.list_gpu_processes())

clear_gpu()

# 🚀 Chạy mô hình XGBoost
dtest = xgb.DMatrix(X_test_np)
y_scores_xgb = xgb_model.predict(dtest)
y_pred_xgb = np.argmax(y_scores_xgb, axis=1)

# 🚀 Chạy mô hình MLP
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)

mlp_model.eval()
y_scores_list, y_pred_list, y_true_list = [], [], []

batch_size = 512
with torch.no_grad():
    for i in range(0, len(X_test_tensor), batch_size):
        X_batch = X_test_tensor[i:i+batch_size]
        Y_batch = Y_test_tensor[i:i+batch_size]

        with torch.amp.autocast(device_type='cuda'):
            logits = mlp_model(X_batch)
            y_scores_batch = F.softmax(logits, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

y_scores_mlp = np.vstack(y_scores_list)
y_pred_mlp = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Kết hợp dự đoán của XGBoost và MLP
y_scores_combined = (y_scores_xgb + y_scores_mlp) / 2
y_pred_combined = np.argmax(y_scores_combined, axis=1)

# 📊 Danh sách nhãn
selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
num_classes = len(selected_classes)

# 📌 Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix - Combined Model')
    plt.show()

# 📌 Vẽ ROC-AUC
def plot_roc_auc(y_true, y_scores, class_names):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 0.02])  # Zoom vào vùng nhỏ để dễ phân biệt
    plt.ylim([0.5, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Zoomed In) - Combined Model')
    plt.legend()
    plt.grid(True)
    plt.show()

# 📌 Vẽ PCA & t-SNE 3D
def plot_tsne_pca_3d(X, y, class_names):
    sample_size = min(5000, len(X))
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")
    colors = cmap(np.linspace(0, 1, len(class_names)))

    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title('PCA 3D - Combined Model')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title('t-SNE 3D - Combined Model')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()

# 📊 Vẽ kết quả cho mô hình kết hợp
plot_confusion_matrix(y_true, y_pred_combined, selected_classes)
plot_roc_auc(y_true, y_scores_combined, selected_classes)
plot_tsne_pca_3d(X_test_np, y_true, selected_classes)

# SAINT****

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 🟢 1️⃣ Load và chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train.values)
X_test_np = scaler.transform(X_test.values)

# 🔴 Đảm bảo lấy đúng dữ liệu nhãn
Y_train_np = Y_train.values
Y_test_np = Y_test.values

# 🟢 2️⃣ Chuyển sang Tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long).squeeze()
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long).squeeze()

# 🟢 3️⃣ Định nghĩa SAINT Model
class SAINT(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64, num_heads=4, num_blocks=3):
        super(SAINT, self).__init__()

        self.embedding = nn.Sequential(
            nn.Linear(num_features, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU()
        )

        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim * 2, dropout=0.1, activation="gelu")
            for _ in range(num_blocks)
        ])

        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Thêm batch dimension cho Transformer
        for block in self.transformer_blocks:
            x = block(x)
        x = x.squeeze(0)  # Bỏ batch dimension
        x = self.classifier(x)
        return x

# 🟢 4️⃣ Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_train_tensor.shape[1]
num_classes = len(torch.unique(Y_train_tensor))  # ✅ Sửa lỗi

model = SAINT(num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# 🟢 5️⃣ Dataloader
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64, shuffle=False)

# 🟢 6️⃣ Huấn luyện với Early Stopping
num_epochs = 10
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    # 🟢 Đánh giá trên tập kiểm tra
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    # 🟢 Early Stopping
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

# 🟢 7️⃣ Đánh giá cuối cùng
print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🚀 Dự đoán trên tập kiểm tra
model.eval()
y_scores_list, y_pred_list, y_true_list = [], [], []

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        outputs = model(X_batch)
        y_scores_batch = F.softmax(outputs, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

y_scores = np.vstack(y_scores_list)
y_pred = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Danh sách nhãn
selected_classes = [f"Class {i}" for i in range(num_classes)]

# 📌 Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix - SAINT Model')
    plt.show()

# 📌 Vẽ ROC-AUC
def plot_roc_auc(y_true, y_scores, class_names):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - SAINT Model')
    plt.legend()
    plt.grid(True)
    plt.show()

# 📌 Vẽ PCA & t-SNE 3D
def plot_tsne_pca_3d(X, y, class_names):
    sample_size = min(5000, len(X))
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")
    colors = cmap(np.linspace(0, 1, len(class_names)))

    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title('PCA 3D - SAINT Model')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title('t-SNE 3D - SAINT Model')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()

# 📊 Vẽ kết quả cho mô hình SAINT
plot_confusion_matrix(y_true, y_pred, selected_classes)
plot_roc_auc(y_true, y_scores, selected_classes)
plot_tsne_pca_3d(X_test_np, y_true, selected_classes)


# DANet (Dual Attention Network)****

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train.values)
X_test_np = scaler.transform(X_test.values)

Y_train_np = Y_train.values
Y_test_np = Y_test.values

# Chuyển thành Tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long)

# Tạo DataLoader
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

# ✅ Định nghĩa mô hình DANet
class DANet(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64):
        super(DANet, self).__init__()

        self.feature_embedding = nn.Linear(num_features, embed_dim)

        # Spatial Attention
        self.spatial_attention = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.Tanh(),
            nn.Linear(embed_dim, 1)
        )

        # Channel Attention
        self.channel_attention = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim),
            nn.Sigmoid()
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.feature_embedding(x)  # (batch_size, embed_dim)

        # Spatial Attention
        attn_weights = self.spatial_attention(x)  # (batch_size, 1)
        attn_weights = torch.softmax(attn_weights, dim=1)
        x = x * attn_weights  # Áp dụng attention weights

        # Channel Attention
        channel_attn = self.channel_attention(x)  # (batch_size, embed_dim)
        x = x * channel_attn  # Áp dụng attention weights

        x = torch.flatten(x, start_dim=1)  # Đảm bảo đúng kích thước

        x = self.classifier(x)  # (batch_size, num_classes)
        return x

# ✅ Khởi tạo mô hình
num_features = X_train_tensor.shape[1]
num_classes = len(np.unique(Y_train_np))

model = DANet(num_features, num_classes).to(device)

# ✅ Định nghĩa loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# ✅ Training loop với Early Stopping
num_epochs = 50
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)  # (batch_size, num_classes)

        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    # ✅ Đánh giá trên tập test
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    # ✅ Early Stopping
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

# ✅ Kết quả cuối cùng
print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🚀 Dự đoán trên tập kiểm tra
model.eval()
y_scores_list, y_pred_list, y_true_list = [], [], []

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        outputs = model(X_batch)
        y_scores_batch = F.softmax(outputs, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

y_scores = np.vstack(y_scores_list)
y_pred = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Danh sách nhãn
selected_classes = [f"Class {i}" for i in range(num_classes)]

# ✅ Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix - DANet Model')
    plt.show()

# ✅ Vẽ ROC-AUC (Chỉ vẽ nếu số lớp > 1)
def plot_roc_auc(y_true, y_scores, class_names):
    if len(class_names) < 2:
        print("⚠️ Không thể vẽ ROC-AUC với chỉ một lớp duy nhất!")
        return

    y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - DANet Model')
    plt.legend()
    plt.grid(True)
    plt.show()

# ✅ Vẽ PCA & t-SNE 3D (Chỉ vẽ nếu số feature > 3)
def plot_tsne_pca_3d(X, y, class_names):
    if X.shape[1] < 3:
        print("⚠️ Không thể vẽ PCA/t-SNE 3D vì số feature quá ít!")
        return

    sample_size = min(5000, len(X))
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")
    colors = cmap(np.linspace(0, 1, len(class_names)))

    # 📌 PCA 3D
    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title('PCA 3D - DANet Model')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    # 📌 t-SNE 3D
    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title('t-SNE 3D - DANet Model')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()

# 📊 Vẽ kết quả cho mô hình DANet
plot_confusion_matrix(y_true, y_pred, selected_classes)
plot_roc_auc(y_true, y_scores, selected_classes)
plot_tsne_pca_3d(X_test_np, y_true, selected_classes)

# TEST CÁC MÔ HÌNH ****

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import joblib  # Dùng để load scaler nếu đã lưu trước đó

# 🔹 Danh sách cột được chọn
selected_features = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate',
    'syn_flag_number', 'psh_flag_number', 'ack_count', 'syn_count', 'fin_count',
    'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'SSH', 'Tot sum', 'Min', 'Max',
    'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius',
    'Covariance', 'Variance', 'Weight'
]

# 🔹 Đọc dữ liệu test
file_path = "/kaggle/input/cic-iot-2023/part-00024-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv"
df_test = pd.read_csv(file_path)

# 🔹 Chỉ giữ lại các cột số
df_test = df_test.select_dtypes(include=["number"])

# 🔹 Kiểm tra cột bị thiếu và điền giá trị trung bình nếu cần
missing_cols = [col for col in selected_features if col not in df_test.columns]
if missing_cols:
    print(f"⚠️ Dữ liệu kiểm tra thiếu các cột: {missing_cols}. Sẽ điền giá trị trung bình!")
    for col in missing_cols:
        df_test[col] = df_test.mean()  # Điền bằng giá trị trung bình

df_test = df_test[selected_features]

# 🔹 Load scaler nếu chưa có
try:
    scaler
except NameError:
    print("⚠️ Chưa có scaler, đang tải từ file...")
    scaler = joblib.load("scaler.pkl")  # Load scaler đã huấn luyện từ trước

# 🔹 Kiểm tra scaler đã huấn luyện chưa
if not hasattr(scaler, "mean_"):
    raise ValueError("⚠️ Bạn chưa huấn luyện `scaler`! Hãy chạy `scaler.fit(data_train)` trước.")

# 🔹 Chuẩn hóa dữ liệu test
X_new_scaled = scaler.transform(df_test)

# 🔹 Chuyển thành DataFrame để giữ tên cột
X_new_scaled_df = pd.DataFrame(X_new_scaled, columns=selected_features)

# 🔹 Chuyển thành Tensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_new_tensor = torch.tensor(X_new_scaled_df.values, dtype=torch.float32, device=device)

# 🔹 Kiểm tra model đã được load chưa
try:
    model
except NameError:
    print("⚠️ Chưa có model, đang tải từ file...")
    model = torch.load("model.pth", map_location=device)  # Load mô hình từ file
    model.eval()

# 🔹 Đảm bảo dữ liệu đầu vào có đúng số chiều
expected_input_size = model.feature_embedding.weight.shape[0]  # Số feature của mô hình
if X_new_tensor.shape[1] != expected_input_size:
    raise ValueError(f"❌ Số lượng features không khớp với model! Cần {expected_input_size}, nhưng dữ liệu test có {X_new_tensor.shape[1]}")

# 🔹 Đánh giá model (giảm batch size để tránh lỗi GPU)
batch_size = 512  # Giảm batch size để tránh lỗi CUDA
num_batches = (X_new_tensor.shape[0] // batch_size) + 1

predicted_labels = []
model.eval()
with torch.no_grad():
    for i in range(num_batches):
        batch = X_new_tensor[i * batch_size: (i + 1) * batch_size]
        if batch.shape[0] == 0:
            continue
        outputs = model(batch)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.cpu().numpy())

# 🔹 Lưu kết quả vào file CSV
df_test["Predicted_Label"] = predicted_labels
df_test.to_csv("predicted_results_fixed.csv", index=False)

print("✅ Kết quả dự đoán đã được lưu vào predicted_results_fixed.csv!")


In [ ]:
import torch
print(torch.cuda.is_available())  # Phải trả về True
print(torch.cuda.device_count())  # Số lượng GPU
print(torch.cuda.get_device_name(0))  # Tên GPU


# **GATv2**

In [ ]:
pip install torch-geometric


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb

# 🟢 1️⃣ Load và chuẩn hóa dữ liệu
def convert_to_dataframe(X):
    """ Chuyển đổi dữ liệu sang pandas DataFrame nếu cần """
    if isinstance(X, torch.Tensor):
        X = X.cpu().numpy()  # Chuyển từ Tensor -> NumPy
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)  # Chuyển từ NumPy -> DataFrame
    return X

X_train = convert_to_dataframe(X_train)
X_test = convert_to_dataframe(X_test)

# 🛠 Kiểm tra xem X_train và X_test có dữ liệu không
if X_train.empty or X_test.empty:
    raise ValueError("❌ X_train hoặc X_test đang bị rỗng!")

# Tìm cột chung giữa X_train và X_test
common_columns = X_train.columns.intersection(X_test.columns)
if len(common_columns) == 0:
    raise ValueError("❌ Không có cột chung nào giữa X_train và X_test!")
X_train = X_train[common_columns]
X_test = X_test[common_columns]

# 🛠 Xử lý giá trị NaN
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

# 🛠 Đảm bảo dữ liệu số
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train)
X_test_np = scaler.transform(X_test)

# Chuyển đổi Y_train và Y_test thành NumPy
def convert_labels(Y):
    """ Chuyển đổi Y thành NumPy """
    if isinstance(Y, torch.Tensor):
        return Y.cpu().numpy()
    return np.array(Y)

Y_train_np = convert_labels(Y_train)
Y_test_np = convert_labels(Y_test)

# 🟢 2️⃣ Huấn luyện XGBoost
dtrain = xgb.DMatrix(X_train_np, label=Y_train_np)
dtest = xgb.DMatrix(X_test_np, label=Y_test_np)

xgb_params = {
    'objective': 'multi:softprob',
    'num_class': len(np.unique(Y_train_np)),
    'max_depth': 6,
    'learning_rate': 0.1,
    'tree_method': 'hist',
    'eval_metric': 'mlogloss'
}

xgb_model = xgb.train(xgb_params, dtrain, num_boost_round=100)

# 🟢 3️⃣ Trích xuất đặc trưng từ XGBoost
X_train_xgb = xgb_model.predict(dtrain)
X_test_xgb = xgb_model.predict(dtest)

X_train_tensor = torch.tensor(X_train_xgb, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_xgb, dtype=torch.float32)

Y_train_tensor = torch.tensor(Y_train_np, dtype=torch.long).squeeze()
Y_test_tensor = torch.tensor(Y_test_np, dtype=torch.long).squeeze()

# 🟢 4️⃣ Định nghĩa MLP
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

# 🟢 5️⃣ Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_train_tensor.shape[1]
num_classes = len(np.unique(Y_train_np))

mlp_model = MLP(num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(mlp_model.parameters(), lr=0.001, weight_decay=1e-4)

# 🟢 6️⃣ DataLoader
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64, shuffle=False)

# 🟢 7️⃣ Huấn luyện MLP
num_epochs = 10
best_test_acc = 0
patience = 5
no_improve_epochs = 0

for epoch in range(num_epochs):
    mlp_model.train()
    train_correct, train_total = 0, 0
    total_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = mlp_model(X_batch)

        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_total += Y_batch.size(0)
        train_correct += (predicted == Y_batch).sum().item()
        total_loss += loss.item()

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_loss / len(train_loader)

    mlp_model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = mlp_model(X_batch)
            _, predicted = torch.max(outputs, 1)
            test_total += Y_batch.size(0)
            test_correct += (predicted == Y_batch).sum().item()

    test_accuracy = 100 * test_correct / test_total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Acc: {test_accuracy:.2f}%")

    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⏹️ Early stopping activated!")
            break

print(f"✅ Final Test Accuracy: {best_test_acc:.2f}%")


In [ ]:
import gc
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize, StandardScaler
import torch.nn.functional as F

# 🖥️ Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🧹 Giải phóng bộ nhớ GPU trước khi chạy
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    print(torch.cuda.list_gpu_processes())

clear_gpu()

# 🚀 Chạy mô hình XGBoost
dtest = xgb.DMatrix(X_test_np)
y_scores_xgb = xgb_model.predict(dtest)
y_pred_xgb = np.argmax(y_scores_xgb, axis=1)

# 🚀 Chạy mô hình MLP
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)

mlp_model.eval()
y_scores_list, y_pred_list, y_true_list = [], [], []

batch_size = 512
with torch.no_grad():
    for i in range(0, len(X_test_tensor), batch_size):
        X_batch = X_test_tensor[i:i+batch_size]
        Y_batch = Y_test_tensor[i:i+batch_size]

        with torch.amp.autocast(device_type='cuda'):
            logits = mlp_model(X_batch)
            y_scores_batch = F.softmax(logits, dim=1).cpu().numpy()

        y_scores_list.append(y_scores_batch)
        y_pred_list.append(np.argmax(y_scores_batch, axis=1))
        y_true_list.append(Y_batch.cpu().numpy())

y_scores_mlp = np.vstack(y_scores_list)
y_pred_mlp = np.concatenate(y_pred_list)
y_true = np.concatenate(y_true_list)

# 📊 Danh sách nhãn
selected_classes = ['DDos', 'DoS', 'Recon', 'Spoofing', 'BruteForce', 'Web-based', 'Mirai', 'BENIGN']
num_classes = len(selected_classes)

# 📌 Vẽ Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()

# 📌 Vẽ ROC-AUC
def plot_roc_auc(y_true, y_scores, class_names, model_name):
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    plt.figure(figsize=(8, 6))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0, 0.02])  # Zoom vào vùng nhỏ để dễ phân biệt
    plt.ylim([0.5, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve (Zoomed In) - {model_name}')
    plt.legend()
    plt.grid(True)
    plt.show()

# 📌 Vẽ PCA & t-SNE 3D
def plot_tsne_pca_3d(X, y, class_names, model_name):
    sample_size = min(5000, len(X))
    idx = np.random.choice(len(X), sample_size, replace=False)
    X_sample = X[idx]
    y_sample = y[idx]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)

    pca = PCA(n_components=3)
    tsne = TSNE(n_components=3, random_state=42)

    X_pca = pca.fit_transform(X_scaled)
    X_tsne = tsne.fit_transform(X_scaled)

    fig = plt.figure(figsize=(14, 6))
    cmap = plt.get_cmap("tab10")
    colors = cmap(np.linspace(0, 1, len(class_names)))

    ax1 = fig.add_subplot(121, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax1.set_title(f'PCA 3D - {model_name}')
    ax1.set_xlabel('PCA 1')
    ax1.set_ylabel('PCA 2')
    ax1.set_zlabel('PCA 3')
    ax1.legend(loc='best')

    ax2 = fig.add_subplot(122, projection='3d')
    for i, class_name in enumerate(class_names):
        mask = (y_sample == i)
        ax2.scatter(X_tsne[mask, 0], X_tsne[mask, 1], X_tsne[mask, 2],
                    color=colors[i], label=class_name, alpha=0.7)
    ax2.set_title(f't-SNE 3D - {model_name}')
    ax2.set_xlabel('t-SNE 1')
    ax2.set_ylabel('t-SNE 2')
    ax2.set_zlabel('t-SNE 3')
    ax2.legend(loc='best')

    plt.show()

# 📌 Vẽ đồ thị Loss & Accuracy
def plot_training_curves(train_loss, val_loss, train_acc, val_acc, model_name):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'r', label='Train Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training & Validation Loss - {model_name}')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'Training & Validation Accuracy - {model_name}')
    plt.legend()

    plt.show()

# 📊 Vẽ kết quả cho từng mô hình
plot_confusion_matrix(y_true, y_pred_xgb, selected_classes, "XGBoost")
plot_confusion_matrix(y_true, y_pred_mlp, selected_classes, "MLP")

plot_roc_auc(y_true, y_scores_xgb, selected_classes, "XGBoost")
plot_roc_auc(y_true, y_scores_mlp, selected_classes, "MLP")

plot_tsne_pca_3d(X_test_np, y_true, selected_classes, "XGBoost")
plot_tsne_pca_3d(X_test_tensor.cpu().numpy(), y_true, selected_classes, "MLP")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

num_features = X_train.shape[1]
num_classes = len(np.unique(Y_train))

class FeatureTokenizer(nn.Module):
    def __init__(self, num_features, embed_dim):
        super().__init__()
        self.linear = nn.Linear(1, embed_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.linear(x)
        cls_tokens = self.cls_token.expand(x.size(0), -1, -1)
        return torch.cat((cls_tokens, x), dim=1)

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.2):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.GELU(),
            nn.Linear(embed_dim * 2, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_output)
        x = self.norm2(x + self.ffn(x))
        return x

class FTTransformer(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64, num_heads=4, num_blocks=3):
        super().__init__()
        self.tokenizer = FeatureTokenizer(num_features, embed_dim)
        self.transformer_blocks = nn.Sequential(
            *[TransformerEncoderBlock(embed_dim, num_heads) for _ in range(num_blocks)]
        )
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.tokenizer(x)
        x = x.permute(1, 0, 2)
        x = self.transformer_blocks(x)
        return self.classifier(x[0])

model = FTTransformer(num_features, num_classes)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (outputs.argmax(1) == Y_batch).sum().item()
        total += Y_batch.size(0)
    train_accuracy = 100 * correct / total
    scheduler.step(total_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f} - Train Acc: {train_accuracy:.2f}%")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        outputs = model(X_batch)
        correct += (outputs.argmax(1) == Y_batch).sum().item()
        total += Y_batch.size(0)
test_accuracy = 100 * correct / total
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

# 5.Gaussian Mixture Model (GMM)

In [ ]:
# Train Gaussian Mixture Model (GMM)
gmm_model = GaussianMixture(n_components=len(np.unique(Y_train)), random_state=42)
gmm_model.fit(X_train)

# Predict and evaluate (GMM requires predicting the class)
gmm_predictions = gmm_model.predict(X_test)
print("GMM Accuracy:", accuracy_score(Y_test, gmm_predictions))
print("GMM Classification Report:\n", classification_report(Y_test, gmm_predictions))

# 6.Gaussian Naive Bayes (GNB)

In [ ]:
# Train Gaussian Naive Bayes (GNB)
gnb_model = GaussianNB()
gnb_model.fit(X_train, Y_train)

# Predict and evaluate
gnb_predictions = gnb_model.predict(X_test)
print("GNB Accuracy:", accuracy_score(Y_test, gnb_predictions))
print("GNB Classification Report:\n", classification_report(Y_test, gnb_predictions))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, Dropout, MaxPooling1D, Flatten
from sklearn.metrics import accuracy_score, classification_report

# Giả sử X_train, X_test, Y_train, Y_test đã được chia và chuẩn hóa trước đó

# Định dạng lại dữ liệu cho CNN & LSTM nếu chưa làm
X_train = np.expand_dims(X_train, axis=2)  # Đảm bảo dữ liệu có dạng (số mẫu, số đặc trưng, 1)
X_test = np.expand_dims(X_test, axis=2)

# Xây dựng mô hình DeepHybrid-IDS (CNN + BiLSTM)
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    Bidirectional(LSTM(64, return_sequences=True)),  # BiLSTM đầu tiên giữ nguyên chuỗi đầu ra
    Bidirectional(LSTM(32)),  # LSTM thứ hai chỉ giữ đầu ra cuối
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(Y_train)), activation='softmax')  # Số lớp đầu ra phù hợp với số nhãn
])

# Compile mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=32)

# Dự đoán và đánh giá mô hình
y_pred = np.argmax(model.predict(X_test), axis=1)

# Đánh giá mô hình
print("DeepHybrid-IDS Accuracy:", accuracy_score(Y_test, y_pred))
print("DeepHybrid-IDS Classification Report:\n", classification_report(Y_test, y_pred, target_names=label_encoder.classes_))


# **Evaluate and Choose Best Model**

In [ ]:
# Create a dictionary to store accuracy of each model
model_accuracies = {
    "Random Forest": accuracy_score(Y_test, rf_predictions),
    "Gradient Boosting": accuracy_score(Y_test, gb_predictions),
    "CNN": cnn_accuracy,
    "SVM": accuracy_score(Y_test, svm_predictions),
    "GMM": accuracy_score(Y_test, gmm_predictions),
    "GNB": accuracy_score(Y_test, gnb_predictions)
}

# Find the model with the highest accuracy
best_model_name = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model_name]

print(f"The best model is: {best_model_name} with accuracy: {best_accuracy * 100:.2f}%")


#  Save the Best Model

In [ ]:
# Save the best performing model
if best_model_name == "Random Forest":
    joblib.dump(rf_model, 'best_rf_model.pkl')
elif best_model_name == "Gradient Boosting":
    joblib.dump(gb_model, 'best_gb_model.pkl')
elif best_model_name == "CNN":
    cnn_model.save('best_cnn_model.h5')
elif best_model_name == "SVM":
    joblib.dump(svm_model, 'best_svm_model.pkl')
elif best_model_name == "GMM":
    joblib.dump(gmm_model, 'best_gmm_model.pkl')
elif best_model_name == "GNB":
    joblib.dump(gnb_model, 'best_gnb_model.pkl')

print(f"Best model saved: {best_model_name}")


In [ ]:
joblib.dump(cnn_model, 'best_cnn_model.pkl')

In [ ]:
import pickle
# Save the entire model as .h5
cnn_model.save('cnn_model.h5')
print("CNN model saved as 'cnn_model.h5'")

# Save the model architecture (JSON) and weights separately for .pkl
model_json = cnn_model.to_json()
with open('cnn_model.json', 'w') as json_file:
    json_file.write(model_json)

# Save weights with the correct filename extension
cnn_model.save_weights('cnn_model.weights.h5')
print("CNN model weights saved as 'cnn_model.weights.h5'")

# Save both JSON and weights path in a .pkl file
with open('cnn_model.pkl', 'wb') as f:
    pickle.dump({'model_json': 'cnn_model.json', 'weights': 'cnn_model.weights.h5'}, f)
print("CNN model structure and weights paths saved in 'cnn_model.pkl'")

In [ ]:
# Load the .pkl file
with open('cnn_model.pkl', 'rb') as f:
    data = pickle.load(f)

# Load model architecture from JSON
with open(data['model_json'], 'r') as json_file:
    cnn_model_json = json_file.read()

# Reconstruct the model and load weights
cnn_model = tf.keras.models.model_from_json(cnn_model_json)
cnn_model.load_weights(data['weights'])
print("CNN model loaded from 'cnn_model.pkl'")